# To Run Standard Searches
- define new searches in mysets.py
- beginning of notebook is assumed to be interactive until the requests have been checked
- all progress and exception logging is done only for main loop

In [1]:
import pandas as pd
import gcsfs
import xarray as xr
from datetime import datetime
import itertools
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
import myconfig
from mysets import all_search
from mydataset import dir2url_, dir2dict
from mytasks import Check, Download, ReadFiles, SaveAsZarr, Upload, Cleanup
from mysearch import esgf_search
from myidentify import gsurl2search
from myutilities import search_df, remove_from_GC_bucket, remove_from_catalogs

In [3]:
# Directly connect to Search Status Page for updating
# Anyone can view: https://docs.google.com/spreadsheets/d/1yAt7604tVt7OXXZUyL2uALtGP2WVa-Pb5NMuTluFsAc/edit?usp=sharing

json_keyfile = '/home/naomi/json/CMIP6-d0cb1df722d1.json'
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
gc = gspread.authorize(credentials)
sheet_name = "CMIP6_UpdateSchedule"
sh = gc.open(sheet_name)
wks = sh.worksheet("Searches")

# read the first row to get the column labels
columns = wks.row_values(3)
col_status = columns.index('status')
col_run = columns.index('last run')
col_drive = columns.index('HD')
col_dataset = columns.index('current dataset')

In [5]:
# CONFIGURE ESGF Search here
node_pref = myconfig.node_pref
dtype = myconfig.dtype
hd = '/h121'
myconfig.local_target_prefix = hd + '/naomi/zarr-minimal/'
dir2local = dir2url_(myconfig.local_target_prefix)

clear_logs = True
update_ESGF = True
NewNeeded = True

search = 'A2d-1e'   # if search='test', must define below

try:
    search_row = wks.find(search).row
    print(search_row)
except:
    print('need another row in Google Form!')
    wks.append_row([search])
    search_row = wks.find(search).row

date = datetime.now().strftime('%Y-%m-%d')
wks.update_cell(search_row, col_run + 1, date)
wks.update_cell(search_row, col_drive + 1, hd)

ESGF_site = dtype['llnl']
#ESGF_site = dtype['dkrz']

print('zarrs will be written to: ',myconfig.local_target_prefix)

zarrs will be written to:  /h121/naomi/zarr-minimal/


In [6]:
# CONFIGURE GCS
fs     = gcsfs.GCSFileSystem(token='anon', access='read_only',cache_timeout=-1)
df_GCS = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv', dtype='unicode')

# make available to all modules (except those)
myconfig.fs = fs
myconfig.df_GCS = df_GCS

In [7]:
if search in all_search:
    # Use a pre-defined search
    asearch = all_search[search]
else:
    # Define your own search: Pick keyword values to specify your search, not specifying a particular keyword means it will find all.
    asearch = {
                     'table_id'      : ['SImon']
                    ,'experiment_id' : ['ssp119', 'ssp126']
                    ,'variable_id'   : ["sithick", "siconc", "sisnthick", "sisnconc"]
                    #,'member_id'     : ['r1i1p1f1']
                    #,'source_id'     : ['CESM2-WACCM']
                }

# check if ANOTHER notebook is doing the same search
lock_file = f'logs/{search}.lock'
if os.path.exists(lock_file):
    if clear_logs:  
        command = f'/bin/rm logs/*{search}*'
        print(command)
        os.system(command)
    else:
        assert False, f'lockfile {lock_file} exists'
        
f = open(lock_file,'w')
date = str(datetime.now().strftime("%Y%m%d-%H%M"))
f.write(f'started {search} at {date}')
f.close()

search, asearch

/bin/rm logs/*A2d-1e*


('A2d-1e',
 {'table_id': ['Amon'],
  'experiment_id': ['1pctCO2',
   'abrupt-4xCO2',
   'historical',
   'piControl',
   'ssp119',
   'ssp126',
   'ssp245',
   'ssp370',
   'ssp434',
   'ssp460',
   'ssp534-over',
   'ssp585'],
  'variable_id': ['tasmax', 'tasmin']})

In [8]:
find_oldversions = True

df_GCS_matching = search_df(df_GCS,**asearch)
if find_oldversions:
    dflcat = df_GCS.copy()

    gsurls = df_GCS_matching.zstore.unique()
    num_removed = 0
    for gsurl in gsurls:
        version_cat = df_GCS[df_GCS.zstore == gsurl].version.values[0] 
        asearch = gsurl2search(gsurl)
        dfs = esgf_search(asearch, toFilter = False)
        try:
            version_ESGF = sorted(list(set(dfs.version_id)))[-1][1:]
        except:
            print('no version currently exists at ESGF',gsurl)
            version_ESGF = 0
        if int(version_cat) < int(version_ESGF):
            print('newer version available:',gsurl)
            print(version_cat,version_ESGF)
            remove_from_GC_bucket(gsurl,execute=True)
            dflcat = dflcat[dflcat['zstore'] != gsurl] 
            num_removed += 1
            
    df_GCS = dflcat
    myconfig.df_GCS = dflcat
    print('number of GC datasets removed = ',num_removed)    

newer version available: gs://cmip6/CMIP/CCCma/CanESM5/1pctCO2/r1i1p1f1/Amon/tasmax/gn/
20190306 20190429
empty search response
no version currently exists at ESGF gs://cmip6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/1pctCO2/r5i1p1f2/Amon/tasmax/gr/
empty search response
no version currently exists at ESGF gs://cmip6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/1pctCO2/r6i1p1f2/Amon/tasmax/gr/
newer version available: gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/1pctCO2/r1i1p1f1/Amon/tasmax/gr/
20190702 20200325
newer version available: gs://cmip6/CMIP/INM/INM-CM5-0/1pctCO2/r1i1p1f1/Amon/tasmax/gr1/
20190820 20200226
newer version available: gs://cmip6/CMIP/CCCma/CanESM5/abrupt-4xCO2/r1i1p1f1/Amon/tasmax/gn/
20190306 20190429
newer version available: gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/abrupt-4xCO2/r1i1p1f1/Amon/tasmax/gr/
20190702 20200225
newer version available: gs://cmip6/CMIP/EC-Earth-Consortium/EC-Earth3/abrupt-4xCO2/r8i1p1f1/Amon/tasmax/gr/
20191020 20200501
newer version available: gs:

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



KeyboardInterrupt: 

In [ ]:
# make dataframe of ESGF search results

if update_ESGF:
    x = [value for key,value in asearch.items()]
    searches = [p for p in itertools.product(*x)]

    dESGF = []
    for s in searches:
        dsearch = dict(zip(asearch.keys(),s))
        print(dsearch)
        df = esgf_search(dsearch, server=ESGF_site)
        if len(df)>0:
            dESGF += [df]

    df_ESGF = pd.concat(dESGF)
    df_ESGF.to_csv(f'csv/ESGF_{search}.csv',index=False)
else:
    df_ESGF = pd.read_csv(f'csv/ESGF_{search}.csv', dtype='unicode')

len(df_ESGF), len(df_ESGF.ds_dir.unique())

In [ ]:
# make dataframe of all needed

if NewNeeded:
    df = pd.merge(df_ESGF,df_GCS, how='outer', indicator=True)
    df_needed = df[df._merge == 'left_only']
    
    keep_keys = df_ESGF.keys()
    all_keys = df.keys()
    drop_keys = list(set(all_keys) - set(keep_keys))
    df_needed = df_needed.drop(drop_keys,1)

    num_stores = 0
    if len(df_needed) > 0:
        num_stores = df_needed.ds_dir.nunique() 
        print(f'needed: nfiles={len(df_needed)}, nstores={num_stores}')
    else:
        print('no new data available')
        exit    
    
    df_needed['member'] = [int(s.split('r')[-1].split('i')[0]) for s in df_needed['member_id']]
    df_needed = df_needed.sort_values(by=['member'])
    #df_needed['zsize'] = [df_needed[df_needed.ds_dir==zs]['file_size'].sum() for zs in df_needed['ds_dir']]
    #df_needed = df_needed.sort_values(by=['zsize'])
    
    df_needed.to_csv(f'csv/needed_{search}.csv',index=False)
else:
    df_needed = pd.read_csv(f'csv/needed_{search}.csv')

print('Variables')
for var in df_needed.variable_id.unique():
    print(var,df_needed[df_needed.variable_id==var].ds_dir.nunique())

print('\nExperiments')
for exp in df_needed.experiment_id.unique():
    print(exp,df_needed[df_needed.experiment_id==exp].ds_dir.nunique())

In [ ]:
# make available to all modules
myconfig.df_needed = df_needed

In [ ]:
ds_dirs = df_needed.ds_dir.unique()
numdsets = len(ds_dirs)

In [ ]:
progress_log  = f'logs/progress_{search}.log'
failure_log   = f'logs/failure_{search}.log'
success_log   = f'logs/success_{search}.log'
logs = {1:progress_log, 2:failure_log, 3:success_log}

### ierr:
- 0 : proceed with next task
- 1 : write to progress_log, go to next dataset (finished or try again)
- 2 : write to failure_log,  go to next dataset (mark as un-usable - do not try again until problem is solved) 
- 3 : write to success_log,  go to next dataset (dataset added to cloud)

In [ ]:
def write_log(file,str,verbose=True):
    f = open(file,'a')
    if verbose:
        print(str)
    f.write(f'{str}\n')
    f.close()
    return

In [ ]:
# reload the catalog  -  incompatible with deleting old versions - don't do
#df_GCS = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores-noQC.csv', dtype='unicode')#
#myconfig.df_GCS = df_GCS

# refresh the gcsfs
fs.invalidate_cache()
date = str(datetime.now().strftime("%Y%m%d-%H%M"))
for log in [1,2,3]:   
    write_log(logs[log],f'starting at {date} \n') 

for item, ds_dir in enumerate(ds_dirs):

    print(f'\n{item}/{numdsets-1}',ds_dir)

    #skip = 'EC-Earth3'
    skip = 'none'
    if skip in ds_dir:
        write_log(progress_log,f'{ds_dir} skipping {skip}') 
        continue
    
    (ierr, exc) = Check(ds_dir, dir2local)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir} {exc}'); continue
                     
    (gfiles, ierr, exc) = Download(ds_dir)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir}, {exc}'); continue
        
    search_row = wks.find(search).row
    wks.update_cell(search_row, col_dataset + 1, ds_dir)
    
    (ds,ierr,exc) = ReadFiles(ds_dir, gfiles, dir2dict)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir}, {exc}'); continue

    (ierr,exc) = SaveAsZarr(ds_dir, ds, dir2local)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir}, {exc}'); continue
        
    (ierr,exc) = Upload(ds_dir, dir2local)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir}, {exc}'); continue

    (ierr,exc) = Cleanup(ds_dir, gfiles, dir2local)
    if ierr > 0:
        write_log(logs[ierr],f'{ds_dir}, {exc}'); continue

    date = datetime.now().strftime('%H:%M, %b%d')
    status_str = f'{item+1} of {numdsets} at {date}'
    search_row = wks.find(search).row
    wks.update_cell(search_row, col_status + 1, status_str)

    write_log(success_log,f'{dir2local(ds_dir)} saved to GCS'); continue